In [2]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

In [1]:
import os
bool(os.environ["OPENAI_API_KEY"])

True

In [3]:
@tool
def get_weather(location: str) -> str:
    '''
    This tool is used to get the weather in a given location in celsius.
    '''
    data = {"chennai": 33, "delhi": 29}
    return f"The weather in {location} is {data.get(location.lower(), 25)} celsius"

@tool
def calculator(expression: str) -> str:
    '''
    This tool is used to evaluate a mathematical expression.
    The input should be a string of a mathematical expression.
    '''
    return str(eval(expression))

@tool
def get_time(timezone: str = "UTC") -> str:
    '''
    This tool is used to get the current time in a given timezone.
    '''
    import datetime
    from pytz import timezone as tz

    now = datetime.datetime.now(tz(timezone))
    return now.strftime("%Y-%m-%d %H:%M:%S")


tool_registry = {
    "get_weather": get_weather,
    "calculator": calculator,
    "get_time": get_time
}

tools = [tool_registry[tool_name] for tool_name in tool_registry]

# Use nano model directly
llm = ChatOpenAI(model="gpt-4.1-nano")
llm_with_tools = llm.bind_tools(tools)

In [4]:
# Initial user message
messages = [HumanMessage(content="What's the weather in Chennai and also add the integers 2,3,5?")]

# Run LLM to get tool calls
ai_msg = llm_with_tools.invoke(messages)
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_guXMyWA4luNHnWmVKEKzT0ZB', 'function': {'arguments': '{"location": "Chennai"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_8QO7z2l6fYZ2MnPqwbcdV88a', 'function': {'arguments': '{"expression": "2+3+5"}', 'name': 'calculator'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 139, 'total_tokens': 188, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_04d3664870', 'id': 'chatcmpl-CJPMGg8iNocyH8EQjlUVDarVgxM5i', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--de485a17-0749-44a9-97bf-d6033793ce73-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Chennai'}, 'id': 'call_guXMy

In [5]:

# Append the AIMessage containing tool_calls
messages.append(ai_msg)

# Run each tool call manually and append ToolMessage
for tool_call in ai_msg.tool_calls or []:
    tool_name = tool_call["name"]
    arguments = tool_call["args"]

    if tool_name in tool_registry:
        # The LangChain guide uses `invoke` directly for the tool call
        tool_msg = tool_registry[tool_name].invoke(tool_call)
    else:
        tool_msg = ToolMessage(
            content="Unknown tool",
            tool_name=tool_name,
            tool_call_id=tool_call["id"],
            tool_input=json.dumps(arguments)
        )

    messages.append(tool_msg)


In [ ]:
# Re-run the LLM with updated context from tool calls
final_response = llm_with_tools.invoke(messages)
final_response

AIMessage(content='The weather in Chennai is 33 degrees Celsius, and the sum of the integers 2, 3, and 5 is 10.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 212, 'total_tokens': 242, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_04d3664870', 'id': 'chatcmpl-CJPMiX5XoWhLtaMicLVUausOwRZcy', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1c3dd526-d902-4c27-941e-ed011d2219e1-0', usage_metadata={'input_tokens': 212, 'output_tokens': 30, 'total_tokens': 242, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
# Using the built-in agent executor
# This is a more compact way to do the same thing as above and is recommended for most use cases
# It is a wrapper around the above code and handles the tool calling and message management

# Agent prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What's the weather in Chennai and also add [2,3,5]?"})



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'location': 'Chennai'}`


The weather in Chennai is 33 celsius
Invoking: `calculator` with `{'expression': '2+3+5'}`


10The weather in Chennai is 33 degrees Celsius, and the sum of [2, 3, 5] is 10.

> Finished chain.


{'input': "What's the weather in Chennai and also add [2,3,5]?",
 'output': 'The weather in Chennai is 33 degrees Celsius, and the sum of [2, 3, 5] is 10.'}